In [1]:
import os
print(os.getcwd())

C:\Users\canno\NBA Props


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from IPython.display import display

# Path to your cross-validation fold files
FOLD_PATH = Path("./cv_folds")

# Hyperparameter grid
param_grid = {
    'RandomForest': [
        {'n_estimators': 100, 'max_depth': None},
        {'n_estimators': 200, 'max_depth': 10},
    ],
    'ExtraTrees': [
        {'n_estimators': 100, 'max_depth': None},
        {'n_estimators': 200, 'max_depth': 10},
    ],
    'GBM': [
        {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 5},
        {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 5},
    ]
}

results = []

for model_name, configs in param_grid.items():
    for config in configs:
        # Instantiate the estimator with current hyperparameters
        if model_name == 'RandomForest':
            estimator = RandomForestRegressor(random_state=42, **config)
        elif model_name == 'ExtraTrees':
            estimator = ExtraTreesRegressor(random_state=42, **config)
        else:  # GBM
            estimator = GradientBoostingRegressor(random_state=42, **config)

        # Build a pipeline: impute missing values → model
        pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('model', estimator)
        ])

        # Store results for each fold and for each target
        fold_results = {
            'fold': [],
            'MAE_pts': [], 'MAE_reb': [], 'MAE_ast': [],
            'RMSE_pts': [], 'RMSE_reb': [], 'RMSE_ast': [],
            'R2_pts': [],  'R2_reb': [],  'R2_ast': []
        }

        # 10-fold cross-validation loop
        for fold_num in range(1, 11):
            train = pd.read_csv(FOLD_PATH / f"fold_{fold_num}_train.csv")
            test  = pd.read_csv(FOLD_PATH / f"fold_{fold_num}_test.csv")

            # Feature selection: only 8-game stats and opponent stats
            initial_feats = [c for c in train.columns if '_8_game_' in c or c.startswith('opp_')]
            numeric_feats = train[initial_feats].select_dtypes(include='number').columns.tolist()

            X_train = train[numeric_feats]
            X_test  = test[numeric_feats]
            y_train = train[['PTS', 'REB', 'AST']]
            y_test  = test[['PTS', 'REB', 'AST']]

            # Fit & predict
            mor   = MultiOutputRegressor(pipeline)
            mor.fit(X_train, y_train)
            preds = mor.predict(X_test)

            # Compute metrics per target
            mae_vals  = mean_absolute_error(y_test, preds, multioutput='raw_values')
            mse_vals  = mean_squared_error(y_test, preds, multioutput='raw_values')  # squared=True by default
            rmse_vals = np.sqrt(mse_vals)                                           # manual RMSE
            r2_vals   = r2_score(y_test, preds, multioutput='raw_values')

            # Store fold metrics
            fold_results['fold'].append(fold_num)
            fold_results['MAE_pts'].append(mae_vals[0])
            fold_results['MAE_reb'].append(mae_vals[1])
            fold_results['MAE_ast'].append(mae_vals[2])
            fold_results['RMSE_pts'].append(rmse_vals[0])
            fold_results['RMSE_reb'].append(rmse_vals[1])
            fold_results['RMSE_ast'].append(rmse_vals[2])
            fold_results['R2_pts'].append(r2_vals[0])
            fold_results['R2_reb'].append(r2_vals[1])
            fold_results['R2_ast'].append(r2_vals[2])

        # Calculate average metrics across folds
        avg_metrics = {metric: np.mean(vals) for metric, vals in fold_results.items() if metric != 'fold'}

        # Record results for this model/config and each fold
        entry = {'model': model_name, **config, **avg_metrics}
        results.append(entry)

        # Display per-fold results for this model/config
        print(f"Results for {model_name} with {config}:")
        fold_df = pd.DataFrame(fold_results)
        display(fold_df)

# Build summary DataFrame and display
summary_df = pd.DataFrame(results)
display(summary_df)


Results for RandomForest with {'n_estimators': 100, 'max_depth': None}:


fold   MAE_pts   MAE_reb   MAE_ast  RMSE_pts  RMSE_reb  RMSE_ast    R2_pts  \
0     1  6.859285  2.325356  2.051648  8.602280  2.882210  2.538739  0.104465   
1     2  5.697096  2.156037  1.709270  7.205041  2.800639  2.309571  0.315288   
2     3  5.594813  2.121064  1.547340  7.078069  2.801038  2.011264  0.304685   
3     4  5.534036  2.193070  1.647184  7.033927  2.822347  2.185364  0.378577   
4     5  5.450081  2.107176  1.641502  6.956027  2.756461  2.143922  0.373709   
5     6  5.544316  2.141852  1.647903  7.083886  2.796336  2.146150  0.390454   
6     7  5.602379  2.189628  1.610124  7.108761  2.846984  2.067423  0.337836   
7     8  5.491177  2.191914  1.688352  7.221813  2.843518  2.226679  0.370080   
8     9  5.408721  2.143633  1.671050  6.807990  2.737261  2.201695  0.336435   
9    10  5.740045  2.268582  1.720946  7.258657  2.906508  2.233900  0.342773   

     R2_reb    R2_ast  
0  0.281073  0.268396  
1  0.374944  0.372773  
2  0.380341  0.452207  
3  0.401733  0.393374  
4  0.440587  0.434018  
5  0.404840  0.424706  
6  0.379533  0.439592  
7  0.358902  0.405822  
8  0.333748  0.387039  
9  0.363051  0.415542

Results for RandomForest with {'n_estimators': 200, 'max_depth': 10}:


fold   MAE_pts   MAE_reb   MAE_ast  RMSE_pts  RMSE_reb  RMSE_ast    R2_pts  \
0     1  6.878153  2.332923  2.038639  8.610233  2.898435  2.530356  0.102809   
1     2  5.618754  2.128951  1.691171  7.119504  2.774306  2.294678  0.331449   
2     3  5.498400  2.086779  1.539929  6.935634  2.759343  2.000523  0.332388   
3     4  5.463941  2.156336  1.619089  6.932443  2.784455  2.156753  0.396379   
4     5  5.352361  2.054408  1.604524  6.821163  2.714810  2.099815  0.397759   
5     6  5.443855  2.120075  1.622459  6.947860  2.759568  2.117406  0.413638   
6     7  5.512565  2.167154  1.564597  6.996536  2.831920  2.022090  0.358578   
7     8  5.378823  2.142586  1.653840  7.066254  2.787468  2.190437  0.396925   
8     9  5.306185  2.091002  1.639722  6.685742  2.678193  2.163838  0.360052   
9    10  5.677806  2.219637  1.682432  7.198430  2.862034  2.204966  0.353634   

     R2_reb    R2_ast  
0  0.272957  0.273220  
1  0.386643  0.380837  
2  0.398651  0.458042  
3  0.417689  0.409154  
4  0.457365  0.457066  
5  0.420388  0.440013  
6  0.386082  0.463898  
7  0.383927  0.425007  
8  0.362192  0.407937  
9  0.382395  0.430583

Results for ExtraTrees with {'n_estimators': 100, 'max_depth': None}:


...

Results for GBM with {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 5}:


fold   MAE_pts   MAE_reb   MAE_ast  RMSE_pts  RMSE_reb  RMSE_ast    R2_pts  \
0     1  7.097457  2.368062  2.103196  8.978599  2.992151  2.679850  0.024399   
1     2  5.676802  2.235329  1.712464  7.123342  2.877540  2.307646  0.330728   
2     3  5.663316  2.176485  1.616773  7.081446  2.872809  2.058408  0.304021   
3     4  5.721104  2.233659  1.671480  7.246228  2.893582  2.199980  0.340499   
4     5  5.630523  2.167064  1.706316  7.051210  2.848534  2.208624  0.356452   
5     6  5.713471  2.190512  1.691598  7.265754  2.858433  2.195454  0.358754   
6     7  5.707709  2.264377  1.648978  7.104987  2.938206  2.118309  0.338539   
7     8  5.677899  2.226010  1.746590  7.376871  2.858627  2.296198  0.342740   
8     9  5.481721  2.163308  1.728995  6.835731  2.743437  2.247979  0.331017   
9    10  5.885324  2.307743  1.784577  7.426939  2.977835  2.312293  0.311945   

     R2_reb    R2_ast  
0  0.225181  0.184806  
1  0.340147  0.373819  
2  0.348179  0.426225  
3  0.371151  0.385232  
4  0.402591  0.399340  
5  0.378113  0.397970  
6  0.339135  0.411665  
7  0.352071  0.368141  
8  0.330739  0.360997  
9  0.331406  0.373802

Results for GBM with {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 5}:


fold   MAE_pts   MAE_reb   MAE_ast  RMSE_pts  RMSE_reb  RMSE_ast    R2_pts  \
0     1  7.225374  2.511652  2.109907  9.184481  3.170957  2.676704 -0.020856   
1     2  5.740240  2.203026  1.740642  7.284615  2.867002  2.372202  0.300080   
2     3  5.544988  2.111406  1.539039  7.042817  2.792259  2.012783  0.311594   
3     4  5.535179  2.171752  1.654302  7.020760  2.803440  2.194034  0.380901   
4     5  5.334977  2.062136  1.601524  6.831401  2.710556  2.109375  0.395949   
5     6  5.474047  2.138870  1.607079  6.994964  2.793030  2.107598  0.405660   
6     7  5.511001  2.208763  1.578164  6.984276  2.890575  2.034839  0.360824   
7     8  5.364514  2.144361  1.643471  7.056773  2.789792  2.188269  0.398543   
8     9  5.268091  2.094127  1.638675  6.654350  2.690280  2.167209  0.366047   
9    10  5.659287  2.229338  1.689698  7.177511  2.873645  2.215317  0.357385   

     R2_reb    R2_ast  
0  0.129811  0.186719  
1  0.344971  0.338294  
2  0.384219  0.451379  
3  0.409722  0.388551  
4  0.459064  0.452111  
5  0.406246  0.445189  
6  0.360387  0.457117  
7  0.382899  0.426145  
8  0.356423  0.406091  
9  0.377374  0.425225

model  n_estimators  max_depth   MAE_pts   MAE_reb   MAE_ast  \
0  RandomForest           100        NaN  5.692195  2.183831  1.693532   
1  RandomForest           200       10.0  5.613084  2.149985  1.665640   
2    ExtraTrees           100        NaN  5.699986  2.214511  1.693521   
3    ExtraTrees           200       10.0  5.545990  2.138072  1.642539   
4           GBM           100        5.0  5.825533  2.233255  1.741097   
5           GBM           200        5.0  5.665770  2.187543  1.680250   

   RMSE_pts  RMSE_reb  RMSE_ast    R2_pts    R2_reb    R2_ast  learning_rate  
0  7.235645  2.819330  2.206471  0.325430  0.371875  0.399347            NaN  
1  7.131380  2.785053  2.178086  0.344361  0.386829  0.414576            NaN  
2  7.239136  2.860770  2.214209  0.325545  0.353225  0.395295            NaN  
3  7.031234  2.768908  2.149986  0.363518  0.393669  0.429813            NaN  
4  7.349111  2.886116  2.262474  0.303909  0.341871  0.368200           0.01  
5  7.223195  2.838154  2.207833  0.325613  0.361112  0.397682           0.05